# Assignment 5v1.0

# Implementing CNN using TensorFlow

Contents :

* Data extraction
* Visualizing the data
* Defining Data Augumentation functions
* Data engineeering (one hot encoding)
* Applying Image transformations
* Defining functons for Convolution, Max pool, ReLU, Flatten and Fully connected layers.
* Setting the sequence of the execution of Layers
* Running the session
* The hyperparameters :
    * Number of Epochs: 100
    * Batch size: 100
    * Number of augumented batches: 6
    * Learning rate: alpha = 1e-4
    * Convolution layers: 3, filter size = 5*5, strides = [1,1,1,1] 
    * Max pool layers: 3, filter size = 2*2, strides = [1,2,2,1]
    * ReLU layers: 3
    * Fully connected layers: 2
    * Flatten layer: 1
* The layer sequence of the architecture: conv1 > pool1 > relu1 > conv2 > pool2 > relu2 > flatten > fc1 > relu3 > fc2
* In 10 epochs:
        * Training accuracy = 0.66
        * Validation accuracy = 0.84

In [1]:
import random   #imports random module, which contains a variety of things to do with random numbers
import skimage as sk   # imports scikit-image library that contains algorithms for image processing
from skimage import transform   # imports transform module from skimage
from skimage import util   # imports util module from skimage; provides utility functions to convert
                           # both dtype and the data range.
from scipy import ndarray   # we use scipy.ndarray to represent the image to transform.
                            # This structure is convinient for computers as it is a 2D array
                            # of image's pixels(RGB colors).
import matplotlib.pyplot as plt   # matplotlib is python 2d plotting library
                                  # pyplot is matplotlib's plotting framework 
%matplotlib inline   
# Turns ON the inline plotting where graphs will be plotted inside the notebook
import copy   # Importing copy module for deep copy operation
import time   # Imports time module that handles various operations regarding time, its conversions and representations.
from sklearn.utils import shuffle   # To shuffle the dataset

In [2]:
# Defining Data Augumentation functions

# A function to add image rotation
def rand_rotation(image_array: ndarray,r = 25):
    # pick a random degree of rotation between r% on the left and r% on the right
    rand_degree = random.uniform(-r,r)
    return sk.transform.rotate(image_array, rand_degree)

# random.uniform(a, b)
# Return a random floating point number N such that a <= N <= b for a <= b and b <= N <= a for b < a.
# The end-point value b may or may not be included in the range depending on floating-point rounding 
# in the equation a + (b-a) * random().

# skimage.transform.rotate(image, angle[, …])
# Rotate image by a certain angle around its center.

# Now a function to add some noise to the image.
def rand_noise(image_array: ndarray):
    return sk.util.random_noise(image_array)

# This function adds random noise of various types to a floating-point image.

# A function to do horizontal flip
def horizontal_flip(image_array: ndarray):
    return image_array[:, ::-1]

# Using Extended slice: Extended slice offers to put a “step” field as [start,stop,step], 
# and giving no field as start and stop indicates default to 0 and string length respectively 
# and “-1” denotes starting from end and stop at the start, hence reversing string.


# Similarly a function to do vertical flip using extended slice will be
def vertical_flip(image_array: ndarray):
    return image_array[::-1, :]



In [3]:
# One hot encoding
n_classes = 10   # Number of classification categories

def one_hot_encoding(z):
    return np.eye(n_classes)[z.astype('int32')]

In [4]:
# import warnings
# warnings.filterwarnings("ignore")
# # Ignore the warnings

In [5]:
# Loading the data

# from mlxtend.data import loadlocal_mnist
# x_train,y_train = loadlocal_mnist(images_path = 'train-images.idx3-ubyte',labels_path = 'train-labels.idx1-ubyte')
# x_test,y_test = loadlocal_mnist(images_path = 't10k-images.idx3-ubyte',labels_path = 't10k-labels.idx1-ubyte')

import numpy as np
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# using keras to load the fashion_mnist dataset

print("x_train shape: ",x_train.shape, "\nx_test shape: ", x_test.shape, "\ny_train shape: ",y_train.shape, "\ny_test shape: ", y_test.shape)

W0620 01:00:44.570425 140377916077888 deprecation_wrapper.py:118] From /home/sayan/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:95: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0620 01:00:44.571241 140377916077888 deprecation_wrapper.py:118] From /home/sayan/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:95: The name tf.AttrValue is deprecated. Please use tf.compat.v1.AttrValue instead.

W0620 01:00:44.571741 140377916077888 deprecation_wrapper.py:118] From /home/sayan/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:95: The name tf.COMPILER_VERSION is deprecated. Please use tf.version.COMPILER_VERSION instead.

W0620 01:00:44.572230 140377916077888 deprecation_wrapper.py:118] From /home/sayan/anaconda3/lib/python3.7/site-packages/tensorflow/__init__.py:95: The name tf.CXX11_ABI_FLAG is deprecated. Please use tf.sysconfig.CXX11_ABI_FLAG instead.

W0620 01:00:44.572745 140377916077888 deprecation_wrapper.py

x_train shape:  (60000, 28, 28) 
x_test shape:  (10000, 28, 28) 
y_train shape:  (60000,) 
y_test shape:  (10000,)


In [6]:
# # Visualizing the dataset

from PIL import Image
Image.fromarray(x_train[0])
Image.fromarray(x_train[80])

# img_index = 100
# plt.imshow(x_train[img_index])   # plots the picture
# plt.show()   # prints the picture that has been plotted

In [7]:
# Data normalization
# We normalize the data dimensions so that they are of approximately the same scale.
# Here we are using min-max normalization.
# x_train.max() = 255 = x_test.max()
# x_train.min() = 0 = x_test.min()

x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [8]:
# import os   # The OS module in python provides functions for interacting with the operating system

# # The nummber of files to generate
# n_files = 

In [9]:
# Image transformation

# Making a dictionary of all the available transformations
available_transformations = {'rotate':rand_rotation, 
                            'noise':rand_noise, 
                            'horizontal_flip':horizontal_flip, 
                            'vertical_flip':vertical_flip}

# We will apply random number of transformations

count = 1
x_train_aug = copy.deepcopy(x_train)   # We perform deep copy so that the initial dataset remains unchanged

max_transformations = 4 #random.randint(1, len(available_transformations))

print("Max. number of transformations done on a single image =", max_transformations)

num_batch = 6   # Number of augumented batches required
batch = 1

aug = None

while batch <= num_batch:
    
    num_transformations = 0
    n_transformations = random.randint(1, max_transformations)   # randint gives random integers
                                                                 # choosing a random number of transformations 
                                                                 # to be apply on a single batch
    while num_transformations < n_transformations:
        
        key = random.choice(list(available_transformations))   # Here we choose the kind of transformations to apply
                                                               # random.choice generates a random sample from 
                                                               # a given 1-D array
        aug = available_transformations[key](x_train)   # The actual transformation process
        num_transformations += 1
        
    x_train_aug = np.vstack((x_train_aug, aug))   # Stacking the training sets together in a row wise fashion   
    print("Batch", batch, "completed.")
    count += 1
    batch += 1    

Max. number of transformations done on a single image = 4
Batch 1 completed.
Batch 2 completed.
Batch 3 completed.
Batch 4 completed.
Batch 5 completed.
Batch 6 completed.


In [10]:
# # Image transformation

# # Making a dictionary of all the available transformations
# available_transformations = {'rotate':rand_rotation, 
#                             'noise':rand_noise, 
#                             'horizontal_flip':horizontal_flip, 
#                             'vertical_flip':vertical_flip}

# n_transformations = 0
# x_train_aug = copy.deepcopy(x_train)   # We perform deep copy so that the initial dataset remains unchanged
# count = 1
# for j in range(len(available_transformations)):
#     n_transformations += 1 
#     # randint gives random integers
#     # choosing a random number of transformations to apply for a single image
#     # print("Max. number of transformations done on a single image =", n_transformations)

#     num_batch = 3   # Number of augumented batches required
#     batch = 1
#     aug = None
    

#     while batch <= num_batch:
    
#         num_transformations = 0    
#         while num_transformations < n_transformations:
        
#             key = random.choice(list(available_transformations))   # Here we choose the kind of transformations to apply
#                                                                    # random.choice generates a random sample from 
#                                                                    # a given 1-D array
#             aug = available_transformations[key](x_train)   # The actual transformation process
#             num_transformations += 1
        
#         x_train_aug = np.vstack((x_train_aug, aug))   # Stacking the training sets together in a row wise fashion   
#         print("Batch", count, "completed.")
#         count += 1
#         batch += 1    

In [11]:
y_train_aug = np.tile(y_train,int(x_train_aug.shape[0]/x_train.shape[0]))   # Cloning the y_train into itself to match 
                                                                          # the corresponding augumented x_train_aug

In [12]:
# One hot encoding of y_train and y_test
y_train_aug = one_hot_encoding(y_train_aug)
y_test = one_hot_encoding(y_test)

In [14]:
# For shuffle
x_train_aug, y_train_aug = shuffle(x_train_aug, y_train_aug, random_state = 0)

In [15]:
x_train_aug.shape, x_train.shape, y_train_aug.shape, y_test.shape, x_test.shape

((420000, 28, 28), (60000, 28, 28), (420000, 10), (10000, 10), (10000, 28, 28))

In [16]:
# A final check of dimensions
assert x_train_aug.shape == (count*x_train.shape[0], 28, 28)
assert y_train_aug.shape == (count*y_train.shape[0], 10)

# Building the CNN

In [17]:
# z = np.reshape(x_train_aug, [-1,14,14,1])
# z.shape, x_train_aug.shape

In [18]:
# Defining placeholder variables for the input images

x = tf.placeholder(tf.float32, shape=[None, 28, 28], name = 'X')

# Reshaping it into [num_images, img_height, img_width, num_channels]
X = tf.reshape(x, [-1, 28, 28, 1])

# Placeholder variables for the true labels associated with the images
Y = tf.placeholder(tf.float32, shape=[None,10], name = 'Y')

In [19]:
X, Y

(<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>,
 <tf.Tensor 'Y:0' shape=(?, 10) dtype=float32>)

In [20]:
# Function for creating a new Convolution Layer

def new_conv_layer(input_dataset, num_input_channels, filter_size, num_filters, name):
    
    with tf.variable_scope(name) as scope:
    # Using "with" we can call anything that calls a context manager. 
    # A context manager provides a simple way to make sure all resources we use are properly cleaned up, 
    # regerdless of if the code returns or an exception is thrown.
    # tf.variable_scope is a context manager for defining ops that creates variable(layers).
    
        # Shape of the filter-weights for the convolution
        shape = [filter_size, filter_size, num_input_channels, num_filters]

        # Create new weights (filters) with the given shape
        weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
        # Outputs random values from a truncated normal distribution.
        # The values whose magnitude is > (2*standard devaitions) from the mean are dropped and re-picked.
        
        # Create new biases, one for each filter
        biases = tf.Variable(tf.constant(0.05, shape=[num_filters]))

        # TensorFlow operation for convolution
        layer = tf.nn.conv2d(input=input_dataset, filter=weights, strides=[1, 1, 1, 1], padding='SAME')
        # This computes a 2D comvolution given a 4D input and filter tensors
        # padding = "SAME" tries to pad evenly left and right, but if the amount of columns to be added is odd, 
        # it will add the extra column to the right.

        # Add the biases to the results of the convolution.
        layer += biases
        
        return layer, weights


In [21]:
y_train_aug.shape

(420000, 10)

In [22]:
# Function for creating a new Pooling Layer

def new_pool_layer(input_dataset, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.max_pool(value=input_dataset, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        # Performs max pooling on the input
        
        return layer


In [23]:
# Function for creating a new ReLU Layer

def new_relu_layer(input_dataset, name):
    
    with tf.variable_scope(name) as scope:
        # TensorFlow operation for convolution
        layer = tf.nn.relu(input_dataset)
        # Computes ReLU operation on the input
        
        return layer

In [24]:
# Function for creating a new Fully connected Layer

def new_fc_layer(input_dataset, num_inputs, num_outputs, name):
    
    with tf.variable_scope(name) as scope:

        # Create new weights and biases.
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev=0.05))
        biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
        
        # Multiply the input and weights, and then add the bias-values.
        layer = tf.matmul(input_dataset, weights) + biases
        
        return layer

In [25]:
# Creating a CNN

# Convolutional Layer 1
layer_conv1, weights_conv1 = new_conv_layer(input_dataset=X, num_input_channels=1, filter_size=5, num_filters=6, name ="conv1")

# Pooling Layer 1
layer_pool1 = new_pool_layer(layer_conv1, name="pool1")

# RelU layer 1
layer_relu1 = new_relu_layer(layer_pool1, name="relu1")

# Convolutional Layer 2
layer_conv2, weights_conv2 = new_conv_layer(input_dataset=layer_relu1, num_input_channels=6, filter_size=5, num_filters=16, name= "conv2")

# Pooling Layer 2
layer_pool2 = new_pool_layer(layer_conv2, name="pool2")

# RelU layer 2
layer_relu2 = new_relu_layer(layer_pool2, name="relu2")

# Flatten Layer
num_features = layer_relu2.get_shape()[1:4].num_elements()
layer_flat = tf.reshape(layer_relu2, [-1, num_features])

# Fully-Connected Layer 1
layer_fc1 = new_fc_layer(layer_flat, num_inputs=num_features, num_outputs=128, name="fc1")

# RelU layer 3
layer_relu3 = new_relu_layer(layer_fc1, name="relu3")

# Fully-Connected Layer 2
layer_fc2 = new_fc_layer(input_dataset=layer_relu3, num_inputs=128, num_outputs=10, name="fc2")

In [26]:
# Using Softmax function to normalize the output
with tf.variable_scope("Softmax"):
    y_pred = tf.nn.softmax(layer_fc2)
#     y_pred_cls = tf.argmax(y_pred, dimension=1)

In [27]:
y_pred#, y_pred_cls

<tf.Tensor 'Softmax/Softmax:0' shape=(?, 10) dtype=float32>

In [28]:
# Using the Cross entropy cost function
with tf.name_scope("cross_ent"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2, labels=Y)
    cost = tf.reduce_mean(cross_entropy)
    
# tf.nn.softmax_cross_entropy_with_logits computes the cross entropy of the function after applying the softmax function

W0620 01:01:29.382840 140377916077888 deprecation.py:323] From <ipython-input-28-9839ac1548b8>:3: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [29]:
cost, cross_entropy

(<tf.Tensor 'cross_ent/Mean:0' shape=() dtype=float32>,
 <tf.Tensor 'cross_ent/softmax_cross_entropy_with_logits_sg/Reshape_2:0' shape=(?,) dtype=float32>)

In [30]:
alpha = 1e-4   # Learning rate for the CNN

In [31]:
# Using Adam Optimizer
with tf.name_scope("optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=alpha).minimize(cost)

In [32]:
optimizer

<tf.Operation 'optimizer/Adam' type=NoOp>

In [33]:
# Accuracy
with tf.name_scope("accuracy"):
    #print(y_pred.shape,Y.shape)
    correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(Y,1))   # Returns the truth value of (y_pred == Y) element-wise.
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    # tf.cast: Casts a tensor to a new type.
    # tf.reduce_mean: Computes the mean of elements across dimensions of a tensor.
#     correct_prediction = tf.equal(y_pred,Y)   # Returns the truth value of (y_pred == Y) element-wise.
#     accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [34]:
accuracy

<tf.Tensor 'accuracy/Mean:0' shape=() dtype=float32>

In [35]:
# Initialize the FileWriter
writer = tf.summary.FileWriter("Training_FileWriter/")
writer1 = tf.summary.FileWriter("Validation_FileWriter/")

In [36]:
# Add the cost and accuracy to summary
tf.summary.scalar('loss', cost)
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

In [37]:
num_epochs = 100
batch_size = 100

# TensorFlow Session

In [38]:
with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    intial_time = time.gmtime()
    
    # Loop over number of epochs
    for epoch in range(num_epochs):
        
        start_time = time.time()   # This function is used to count the number of seconds elapsed since the epoch.
        train_accuracy = 0
        i = 1
        for batch in range(0, int(len(x_train_aug)/batch_size)):
            
            # Get a batch of images and labels
            x_batch = x_train_aug[((i-1)*batch_size):(i*batch_size),:,:]
            y_true_batch = y_train_aug[((i-1)*batch_size):(i*batch_size), :]
            
            # Put the batch into a dict with the proper names for placeholder variables
            feed_dict_train = {x: x_batch, Y: y_true_batch}
            
            # Run the optimizer using this batch of training data.
            sess.run(optimizer, feed_dict=feed_dict_train)
            
            # Calculate the accuracy on the batch of training data
            train_accuracy += sess.run(accuracy, feed_dict=feed_dict_train)
            
            # Generate summary with the current batch of data and write to file
            summ = sess.run(merged_summary, feed_dict=feed_dict_train)
            writer.add_summary(summ, epoch*int(len(x_train_aug)/batch_size) + batch)
            
            i += 1
        
          
        train_accuracy /= int(len(x_train_aug)/batch_size)   # The training accuracy is obtained by dividing 
                                                             # the result by the number of epochs
        
        # Generate summary and validate the model on the entire validation set
        summ, vali_accuracy = sess.run([merged_summary, accuracy], feed_dict={x:x_test, Y:y_test})
        writer1.add_summary(summ, epoch)
        

        end_time = time.time()
        closing_time = time.gmtime()
        
        print("Epoch "+str(epoch+1)+" completed : Time usage "+str(int(end_time-start_time))+" seconds")
        print("\tAccuracy:")
        print ("\t- Training Accuracy:\t{}".format(train_accuracy))
        print ("\t- Validation Accuracy:\t{}".format(vali_accuracy))

Epoch 1 completed : Time usage 144 seconds
	Accuracy:
	- Training Accuracy:	0.5380809496751143
	- Validation Accuracy:	0.6840999722480774
Epoch 2 completed : Time usage 144 seconds
	Accuracy:
	- Training Accuracy:	0.6074666659888767
	- Validation Accuracy:	0.7425000071525574
Epoch 3 completed : Time usage 144 seconds
	Accuracy:
	- Training Accuracy:	0.6282833331184727
	- Validation Accuracy:	0.7771999835968018
Epoch 4 completed : Time usage 144 seconds
	Accuracy:
	- Training Accuracy:	0.6400142864528157
	- Validation Accuracy:	0.7975000143051147
Epoch 5 completed : Time usage 157 seconds
	Accuracy:
	- Training Accuracy:	0.6479904770993051
	- Validation Accuracy:	0.8108999729156494
Epoch 6 completed : Time usage 153 seconds
	Accuracy:
	- Training Accuracy:	0.6534023821708702
	- Validation Accuracy:	0.8230999708175659
Epoch 7 completed : Time usage 156 seconds
	Accuracy:
	- Training Accuracy:	0.6580880963518506
	- Validation Accuracy:	0.8330000042915344
Epoch 8 completed : Time usage 152

W0620 01:31:48.473259 140377916077888 deprecation_wrapper.py:118] From /home/sayan/anaconda3/lib/python3.7/site-packages/tensorflow_estimator/python/estimator/api/_v1/estimator/__init__.py:10: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.



KeyboardInterrupt: 

In [ ]:
print("Final Accuracy after " + str(epoch+1) + " Epochs:")
print ("- Training Accuracy:\t{}".format(train_accuracy))
print ("- Validation Accuracy:\t{}".format(vali_accuracy))
print ("/nTotal time to run the session = " + str(initial_time-closing_time))

In [ ]:
sess.close()